# Ladění parametrů simulace v Gromacsu jako nástroj pro jazykový model

In [ ]:
# tohle všechno potřebujeme ke štěstí
%pip install -r requirements.txt

In [ ]:
# balík gromacs potřebuje binárku `gmx`, použijeme wrapper, který si gromacs spustí ve vedlejším kontejneru v K8s
%cp gmx-wrap2.sh /usr/local/bin/gmx
!chmod +x /usr/local/bin/gmx

In [ ]:
# wrapper gromacsu potřebuje kubectl
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
!chmod +x kubectl
%cp kubectl /usr/local/bin

In [ ]:
import gromacs as gmx
import requests
import json
import time
import pandas as pd
from IPython.display import display, clear_output

## Zvolte si oblíbenou molekulu
Defaultně nastavený `1l2y` je oblíbený pokusný králík, miniprotein "tryptophanová klec", v simulacích se rychle něco děje, ale přitom už jsou realistické.

Dejte si sem, co chcete, ale počítejte, že soubor stažený z databáze musí bez řečí zpracovat `gmx pdb2gmx`, což není pro všechny záznamy v PDB zdaleka zaručeno. 

In [ ]:
mol='1l2y'

## Externí služba ladiče výpočtů
Při zavolání API spustí sadu experimentů v různých konfiguracích a postupně reportuje výsledky. Viz [Adamova diplomka](https://is.muni.cz/auth/th/y6w2x/).

Je třeba zadat endpoint, kde API poslouchá, a přístupové údaje k němu uložit do souboru `tuner-auth`.

In [ ]:
tuner = "https://gromacs-tuner.dyn.cloud.e-infra.cz/api"

with open('tuner-auth') as a:
    tuner_auth = a.read().strip()


## Převzatý tutorial
Víceméně "as is" okopírovaný [playground.ipynb](playground.ipynb), s přizpůsobeným system promptem.

Klíč k OpenAI API očekává v souboru `key`.

In [ ]:
from openai import OpenAI
import json

In [ ]:
#API key
with open('key') as k:
    OPENAI_API_KEY=k.read().strip()

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
SYSTEM_PROMPT="""
You are an assistant who can help the user to find out what are the best parameters like number of MPI processes, number of OMP threads, and GPU use for different
subtasks of molecular dynamics simulation using Gromacs on a particular protein which structure can be downloaded from the PDB database.
Assume the user quite expert in computational biochemistry but he/she does not understand the details of computational setup, and uses this as a blackbox,
just copy and pasting Gromacs command line parameters.
"""

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"What is the best way to run Gromacs molecular dynamics simulation on PDB id {mol}" }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

In [ ]:
print(response.choices[0].message.content)

### Konstrukce vstupu laděného výpočtu
Pokud to někoho zajímá, vychází z proflákuntého tutorialu [Lysosome in water](http://www.mdtutorials.com/gmx/lysozyme/index.html).

Postupně se spustí:
1. Stažení souboru z PDB
2. Konverze do základních vstupních souborů pro Gromacs, specifikace silového pole, velikosti a tvaru simulačního boxu, a modelu vody
3. Osolit, opepřit -- přidáváme ionty sodíku a chloru tak, aby se náboj celého systému vyrovnal na nulu
4. Rychlá energetická minimalizace -- systém stažený z PDB nemusí být v minimálním stavu vůči použitému silovému poli a mohl by dostat na začátku kopanec, proto ho necháme opatrně sklouznout do stabilního stavu
5. Ekvilibrace -- atomy proteinu držíme na místě, vodu "ohřejeme" na 300K (tj. jejím molekulám přidáme rychlosti) a krátkou simulací to necháme sednout do realistického, dynamicky stabilizovaného stavu
6. Vygenerování vstupu pro produkční simulaci -- jediného souboru `md.tpr`, kde je vše

In [ ]:
def getPDB(pdbid):
    url = f"https://files.rcsb.org/download/{pdbid.upper()}.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        with open(f"{pdbid}.pdb", "w") as f:
            f.write(response.text)
        print(f"Downloaded {pdbid}.pdb")
    else:
        raise ValueError(response.text)

In [ ]:
#getPDB(mol)

In [ ]:
def gmxsetup(name):
    gmx.pdb2gmx(f=name+'.pdb',o=name+'.gro',water='tip3p',ff='amber99',p=name+'.top',ignh=True)
    gmx.editconf(f=name+'.gro',o=name+'-box.gro',c=True,d='1.5', bt='dodecahedron')
    !cp {name}.top {name}-solv.top
    gmx.solvate(cp=name+'-box.gro',cs='spc216.gro',o=name+'-solv.gro',p=name+'-solv.top')
    

In [ ]:
#gmxsetup(mol)

In [ ]:
def gmxions(name):
    with open('ions.mdp','w') as ions:
        ions.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = cutoff    ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")
    
    gmx.grompp(f='ions.mdp',c=name+'-solv.gro',p=name+'-solv.top',o='ions.tpr')
    gmx.select(s=name+'-solv.gro',on='solv.ndx',select='SOL')
    !cp {name}-solv.top {name}-ions.top
    gmx.genion(s='ions.tpr',n='solv.ndx',o=name+'-ions.gro',p=name+'-ions.top',pname='NA',nname='CL',neutral=True)

In [ ]:
#gmxions(mol)

In [ ]:
def gmxmin(name):
    with open('minim.mdp','w') as m:
        m.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.005          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout                 = 50         
nstvout                 = 0        
nstfout                 = 0
nstenergy               = 50         
""")
    
    gmx.grompp(f='minim.mdp',c=name+'-ions.gro',p=name+'-ions.top',o='em.tpr')
    gmx.mdrun(deffnm='em')

In [ ]:
#gmxmin(mol)

In [ ]:
# works for small proteins but can be a bit unrealistic (20ps)

def gmxequilib(name,eqsteps = 10000):
    with open('nvt.mdp','w') as nvt:
        nvt.write(f'''title                   = NVT equilibration 
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = {eqsteps}     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = no        ; first dynamics run
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is off
pcoupl                  = no        ; no pressure coupling in NVT
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = yes       ; assign velocities from Maxwell distribution
gen_temp                = 300       ; temperature for Maxwell distribution
gen_seed                = -1        ; generate a random seed
''')
    gmx.grompp(f="nvt.mdp",c="em.gro",r="em.gro",p=name+'-ions.top',o="nvt.tpr")
    gmx.mdrun(deffnm='nvt',pin='on')
    
    with open('npt.mdp','w') as npt:
        npt.write(f'''define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = {eqsteps}     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = yes       ; Restarting after NVT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
; ljocha pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupl = C-rescale
pcoupltype              = isotropic             ; uniform scaling of box vectors
; ljocha tau_p                   = 2.0                   ; time constant, in ps
tau_p = 5.0
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
refcoord_scaling        = com
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
''')
    gmx.grompp(f="npt.mdp",c="nvt.gro",r="nvt.gro",p=name+'-ions.top',o="npt.tpr")
    gmx.mdrun(deffnm='npt',pin='on')

    

In [ ]:
#gmxequilib(mol)

In [ ]:
def gmxmd(name,nsteps = 20*500):
    with open('md.mdp','w') as mdp:
        mdp.write(f'''integrator              = md        ; leap-frog integrator
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
compressed-x-grps       = Protein    
; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300 300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
nsteps = {nsteps}
''')

    gmx.grompp(f="md.mdp",c="npt.gro",r="npt.gro",p=name+'-ions.top',o=name+".tpr")


In [ ]:
# gmxmd(mol)

### Práce s API tuneru
Klasický REST:
- `POST /tuner_runs/` akceptuje soubor `.tpr`, vrátí `ID` experimentu a spustí ladění
- `GET /tuner_runs/ID/status` vrací stav včetně průběžných výsledků ladění
- `DELETE /tuner_runs/ID` pozabíjí případně ještě běžící pokusy a uklidí

V `tunerWatch()` se opakovaně ptáme na stav dokud dost pokusů nedoběhne do konce

In [ ]:
def tunerSubmit(tpr):
    auth = tuple(tuner_auth.split(':'))
    with open(tpr,'rb') as t:
        resp = requests.post(tuner + '/tuner_runs',files={'file':t},auth=auth)
    return resp.json()['data']['tuner_run_id']

In [ ]:
def tunerStatus(id):
    auth = tuple(tuner_auth.split(':'))
    stat = requests.get(tuner + '/tuner_runs/' + id + '/status',auth=auth)
    return stat.json()

In [ ]:
def tunerWatch(id,mindone=6):
    while True:
        stat=tunerStatus(id)
        if stat['success'] and len(stat['data']['trials']) > 0:
            break
            
    while True:
        stat=tunerStatus(id)
        f=pd.DataFrame(stat['data']['trials']).sort_values('performance',ascending=False)
        clear_output()
        display(f)
        if f['status'].value_counts().get('TERMINATED',0) >= mindone:
            break
        time.sleep(10)   

### Funkce nástroje pro jazykový model
Tady je všechno zabalené dohromady spolu s nápovědou pro jazykový model, co funkce dělá a jak a k čemu ji má použít. Cool.

In [ ]:
def gmxtune(mol):
    """
    Download initial PDB structure, set up Gromacs simulation and, by talking to the tuner service, find out what are the most promising
    combinations of parameters like number of MPI processes, number of OMP threads, and GPU use for different
    subtasks of molecular dynamics.

    Args:
        mol (str): The identifier of the input structure to be found in PDB database
    
    Returns:
        str: List of the most promising gromacs commands to execute, including the estimated performance (the more, the better)
    """

    getPDB(mol)
    gmxsetup(mol)
    gmxions(mol)
    gmxmin(mol)
    gmxequilib(mol)
    gmxmd(mol)

    id = tunerSubmit(mol+'.tpr')
    tunerWatch(id)
    
    stat=tunerStatus(id)
    f=pd.DataFrame(stat['data']['trials']).sort_values('performance',ascending=False)
   
    answer = "\n".join([f"mpirun -np {r['np']} gmx mdrun -deffnm md -ntomp {r['ntomp']} -nb {r['nb']} -pme {r['pme']} # {r['performance']} ns/day"
        for _,r in f[f['status']=='TERMINATED'].sort_values('performance',ascending=False).iterrows() ])

    auth = tuple(tuner_auth.split(':'))
    requests.delete(tuner + '/tuner_runs/' + id,auth=auth)

    return answer
    
    # XXX: cheat:
    #return f"""gmx mdrun -ntomp 2 -ntmpi 4 -nb gpu -pme gpu -deffnm {pdbid} # 222 ns/day
#gmx mdrun -ntomp 4 -ntmpi 4 -pme gpu -deffnm {pdbid} 188 ns/day
#"""

### Pokračování tutoriálu 

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": gmxtune.__name__,
        "description": gmxtune.__doc__,
        "parameters": {
            "type": "object",
            "properties": {
                "mol": {"type": "string"},
            },
            "required": ["mol"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
pdbid = eval(response.choices[0].message.tool_calls[0].function.arguments)['mol']
pdbid

In [ ]:
messages.append(
    {
    "role": "tool",
    "tool_call_id": response.choices[0].message.tool_calls[0].id,
    "content": gmxtune(pdbid)
    })
messages

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response

In [ ]:
print(response.choices[0].message.content)

## Manual cleanup

In [ ]:
auth = tuple(tuner_auth.split(':'))
resp=requests.get(tuner + '/tuner_runs',auth=auth)
resp

In [ ]:
resp.json()

In [ ]:
for id in resp.json()[